In [ ]:
import pickle
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
# load training.pkl
path = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Datasets YouTube/training_data.pickle'
with open(path, 'rb') as f:
  training = pickle.loads(f.read())

In [ ]:
# load test.pkl
path = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Datasets YouTube/test_data.pickle'
with open(path, 'rb') as f:
  test = pickle.loads(f.read())

In [ ]:
# load the nlp moduele
import en_core_web_sm

nlp = en_core_web_sm.load(disable=["parser", "tagger", "ner"])

In [ ]:
# do lower-case and remove numbers
def my_cleaner3(text,nlp):

    return[token.lemma_.lower() for token in nlp(text) if not (token.is_alpha==False or len(token.lemma_) <3) ]

In [ ]:
def cleaner(training_data,nlp,filepath,filename):
    '''

    :param training_data: input here your list containing training or test dataset
    :param nlp: specify variable that holds nlp model
    :param filepath: input here your folder where you want the pickle
    :param filepath: input here your name for the pickle

    :return: returns cleaned version (tokenized and lowercased)
    '''

    training_data_cleaned = []
    for i, transcript in enumerate(training_data):
        doc = nlp(training_data[i][0])
        #print("doc: ", doc)
        cleaned_tokens1 = my_cleaner3(doc.text, nlp=nlp)
        #print("cleaned_tokens1: ", cleaned_tokens1)
        training_data_cleaned.append([cleaned_tokens1, training_data[i][1]])
        #print("training_data_cleaned: ", training_data_cleaned)

    import os
    os.chdir(filepath)
    import pickle
    with open(filename+'.pickle', 'wb') as f:
        pickle.dump(training_data_cleaned, f)
    return training_data_cleaned


In [ ]:
# clean all data and pickle them
training_data_cleaned = cleaner(training, nlp,'/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Cleaned data 1', 'training_data_cleaned')
test_data_cleaned = cleaner(test, nlp,'/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Cleaned data 1', 'test_data_cleaned')

After cleaning the raw dataset the data can be loaded with the following code:

In [ ]:
# open training_data_cleaned & test_data_cleaned without making them first 
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Cleaned data 1'

import os
os.chdir(filepath)
with open('training_data_cleaned.pickle', 'rb') as f: 
  training_data_cleaned = pickle.load(f)
with open('test_data_cleaned.pickle', 'rb') as f: 
  test_data_cleaned = pickle.load(f)

In [ ]:
# check
print(len(training_data_cleaned))
print(len(test_data_cleaned))

In [ ]:
# we determine some hyperparameters here for the rest of the research, here we can change them to see the influence of certain model parameters
# as wel as set some key variables
keyword_number = [20, 50, 100, 200, 500, 750, 1000]
SVM_C = [1, 3, 5, 10]

In [ ]:
def keywordextractor(filepath_trans, filepath_togo, filename,keywords_num):

    """ this function creates two picklefiles, each has a keyword list
    first parameter; filepath = the path where you want the pickles to go, and the path where your transcript data is
    filename: name of the pickle: for example; 'cleaned_transcript_data.pickle'
    second parameter; keywords_num = amount of keywords you want in your list"""

    import os
    import pickle
    import numpy as np

    os.chdir(filepath_trans)
    with open(filename, 'rb') as f:
        transcripts_cleaned = pickle.load(f)


    keywords_number = keywords_num
    ### taking only transcripts
    list_document_tokens = []
    for i, document in enumerate(transcripts_cleaned):
        list_document_tokens.append(transcripts_cleaned[i][0])

    # create a list of documents to input to tfidfvectorizer
    tfidf_input = []
    for document in list_document_tokens:
        tfidf_input.append(" ".join(document))

    ### split it by class
    list_document_tokens_consp = []
    list_document_tokens_nonconsp = []
    for i, document in enumerate(transcripts_cleaned):
      #print("i: ", i)
      #print("document: ", document)
      if document[1] == '1':

          list_document_tokens_nonconsp.append(transcripts_cleaned[i][0])
      else:
          list_document_tokens_consp.append(transcripts_cleaned[i][0])

    tfidf_input_consp = []
    tfidf_input_nonconsp = []
    for document in list_document_tokens_consp:
        tfidf_input_consp.append(" ".join(document))
    for document in list_document_tokens_nonconsp:
        tfidf_input_nonconsp.append(" ".join(document))

    # now for keyword extraction method 1

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.feature_extraction.text import CountVectorizer

    tv = TfidfVectorizer(stop_words=None, max_features=10000)

    vocab = tv.fit(tfidf_input)
    feature_names = vocab.get_feature_names()

    word_count_vector = tv.fit_transform(tfidf_input).toarray()

    word_count_vector_transposed = word_count_vector.T

    total_word_info = []
    for i, word in enumerate(word_count_vector_transposed):
        tempword = word.tolist()
        word_info = []

        for j, document in enumerate(tempword):
            word_info.append([j, document, transcripts_cleaned[j][1]])
        total_word_info.append(word_info)

    tf_sum_consp = 0
    tf_sum_nonconsp = 0
    sum_array = []

    for i, word_info in enumerate(total_word_info):
        tf_sum_consp = 0
        tf_sum_nonconsp = 0
        tf_sum_delta = 0

        for array in word_info:
            boolchecker = array[2]

            if boolchecker == 1:

                value = array[1]
                tf_sum_nonconsp += value

            else:
                value = array[1]

                tf_sum_consp += value

        tf_sum_delta = tf_sum_nonconsp - tf_sum_consp

        sum_array.append([feature_names[i], tf_sum_delta])

    deltas = []
    for item in sum_array:
        deltas.append(item[1])

    deltas = np.array(deltas)
    indices = deltas.argsort()[:keywords_number]

    keywords_1 = [sum_array[i] for i in indices]

    keyword_list1 = []
    for i in keywords_1:
        keyword_list1.append(i[0])

    print("there are this many keywords in list1: ", len(keyword_list1))
    # we pickle it for posterity
    os.chdir(filepath_togo)

    with open(str(keywords_num)+'keyword_list1.pickle', 'wb') as f:
        pickle.dump(keyword_list1, f)

    ### now for keyword extraction method 2

    # method two runs a basic pipeline with a SVM then finds most distinguishing features

    os.chdir(filepath_trans)
    with open(filename, 'rb') as f:
        transcripts_cleaned = pickle.load(f)
    print("fully loaded")

    list_document_tokens = []
    for i, document in enumerate(transcripts_cleaned):
        list_document_tokens.append(transcripts_cleaned[i][0])

    tfidf_input = []
    for document in list_document_tokens:
        tfidf_input.append(" ".join(document))

    # now for feature extraction

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer

    tv = TfidfVectorizer(stop_words=None, max_features=10000)
    word_count_vector = tv.fit_transform(tfidf_input)

    tf_idf_vector = tv.fit_transform(tfidf_input).toarray()

    # create X and y

    X = tf_idf_vector
    y = []

    # merge categories
    for document in transcripts_cleaned:

        class_made = 0
        if document[1] == 1:
            class_made = 0
        else:
            class_made = 1
        y.append(class_made)

    # train test split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Support Vector Machine Classifier
    from sklearn import svm
    classifier3 = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    classifier3.fit(X_train, y_train)

    y_pred3 = classifier3.predict(X_test)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


    coef = classifier3.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-int((keywords_number)):]
    top_negative_coefficients = np.argsort(coef)[:int(keywords_number)]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    feature_names = np.array(tv.get_feature_names())
    keyword_list2 = feature_names[top_positive_coefficients]

    print(keyword_list1)
    print(keyword_list2)

    # we pickle it for posterity
    os.chdir(filepath_togo)

    with open(str(keywords_num)+'keyword_list2.pickle', 'wb') as f:
        pickle.dump(keyword_list2, f)

    print("finished extracting keywords")

In [ ]:
# # # # KEYWORD EXTRACTION creates two pickle files of keyword top keywords
filepath_trans = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Cleaned data 1'
filepath_togo = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
for k in keyword_number: 
  keywordextractor(filepath_trans, filepath_togo, 'training_data_cleaned.pickle', k)

This code is to load all the keyword lists once they are created:

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
os.chdir(filepath)
import pickle

keyword = 20
with open(str(keyword) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1_20 = pickle.load(f)
with open(str(keyword) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2_20 = pickle.load(f)

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
os.chdir(filepath)
import pickle

keyword = 50
with open(str(keyword) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1_50 = pickle.load(f)
with open(str(keyword) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2_50 = pickle.load(f)

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
os.chdir(filepath)
import pickle

keyword = 100
with open(str(keyword) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1_100 = pickle.load(f)
with open(str(keyword) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2_100 = pickle.load(f)

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
os.chdir(filepath)
import pickle

keyword = 200
with open(str(keyword) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1_200 = pickle.load(f)
with open(str(keyword) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2_200 = pickle.load(f)

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
os.chdir(filepath)
import pickle

keyword = 500
with open(str(keyword) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1_500 = pickle.load(f)
with open(str(keyword) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2_500 = pickle.load(f)

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
os.chdir(filepath)
import pickle

keyword = 750
with open(str(keyword) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1_750 = pickle.load(f)
with open(str(keyword) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2_750 = pickle.load(f)

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1'
os.chdir(filepath)
import pickle

keyword = 1000
with open(str(keyword) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1_1000 = pickle.load(f)
with open(str(keyword) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2_1000 = pickle.load(f)

Transform the cleaned dataset into a vectorized list

In [ ]:
# Feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# for the train set I create X and y
X_trainingdata = []
y_trainingdata = []
for item in training_data_cleaned:
    X_trainingdata.append(item[0])
    y_trainingdata.append(item[1])

In [ ]:
# Here I transform X to fit the input of the tfidf vectorizer
list_document_tokens = []
for i, document in enumerate(X_trainingdata):
    list_document_tokens.append(training_data_cleaned[i][0])
tfidf_input = []
for document in list_document_tokens:
    tfidf_input.append(" ".join(document))

In [ ]:
# I do the same for my test data
X_testdata = []
y_testdata = []
for item in test_data_cleaned:
    X_testdata.append(item[0])
    y_testdata.append(item[1])

In [ ]:
# once again making them fit
list_document_tokens_test = []
for i, document in enumerate(X_testdata):
    list_document_tokens_test.append(test_data_cleaned[i][0])
tfidf_input_test = []
for document in list_document_tokens_test:
    tfidf_input_test.append(" ".join(document))

The next step is to load and train the BERT model. Once the model is pretrained on the NER-labeled dataset, BERT can predict NER tags on the keyword lists. 

**Clone Repository:**
First Step is to clone KAMALKRAJ Github Repository. Below code is command to clone repository:

In [ ]:
!git clone https://github.com/kamalkraj/BERT-NER.git

Use "ls -l" command for verfying the repository cloned properly.

In [ ]:
ls -l

Now go to 'BERT-NER' directory by using below command:

In [ ]:
cd BERT-NER/

**BERT-NER files:** Use 'ls -l' to check content inside BERT-NER folder. These below files and folders we will use for finetuning and prediction.

In [ ]:
ls -l

"requirements.txt" contains all the pacakages that required for trainig and inference. By using below command install all the pacakages.

In [ ]:
!pip3 install -r requirements.txt

### **Fine-Tuning:**
For finetuning or training the **bert-base** model run the 'run_ner.py' as command given below.

In below command we have to pass different arguments:
   
*   '--data_dir' argument required to collect dataset. Pass 'data/' as argument which we can see as directory inside 'BERT-NER' folder for the previous comment and command for 'BERT-NER files' .
*   '--bert_model' used to download **pretrained bert base** model of Hugging Face transformers. There are different model-names as suggested by hugging face for argument, here we select 'bert-base-cased'.
*   '--task_name' argument used for task to perform. Enter 'ner' as we will train the model for Named Entity Recogintion(NER).
*   '--output_dir' argument is for where to store fine-tuned model. We give name 'out_base' for directory  where fine-tuned model stored.
*   Other arguments like '--max_seq_length', '--num_train_epochs' and '--warmup_proportion', just give values as suggested in repository.
*   For training pass argument '--do_train' and after that evaluating for results pass argument '--do_eval'.





    

In [ ]:
!python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_ner --max_seq_length=128 --do_train --num_train_epochs 3 --do_eval --warmup_proportion=0.1

**Overwrite 'bert.py' files:**
In 'bert.py' we have made changes for better representation and display of '**entity detected**' and their '**entity types**' for the given sentence to test or inference.
>Overwrite the 'bert.py' by using below command.

In [ ]:
%%writefile bert.py
"""BERT NER Inference."""

from __future__ import absolute_import, division, print_function

import json
import os

import torch
import torch.nn.functional as F
from nltk import word_tokenize
from pytorch_transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)


class BertNer(BertForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, valid_ids=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask, head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda' if torch.cuda.is_available() else 'cpu')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)
        return logits

class Ner:

    def __init__(self,model_dir: str):
        self.model , self.tokenizer, self.model_config = self.load_model(model_dir)
        self.label_map = self.model_config["label_map"]
        self.max_seq_length = self.model_config["max_seq_length"]
        self.label_map = {int(k):v for k,v in self.label_map.items()}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.model.eval()

    def load_model(self, model_dir: str, model_config: str = "model_config.json"):
        model_config = os.path.join(model_dir,model_config)
        model_config = json.load(open(model_config))
        model = BertNer.from_pretrained(model_dir)
        tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=model_config["do_lower"])
        return model, tokenizer, model_config

    def tokenize(self, text: str):
        """ tokenize input"""
        words = word_tokenize(text)
        tokens = []
        valid_positions = []
        for i,word in enumerate(words):
            token = self.tokenizer.tokenize(word)
            tokens.extend(token)
            for i in range(len(token)):
                if i == 0:
                    valid_positions.append(1)
                else:
                    valid_positions.append(0)
        return tokens, valid_positions

    def preprocess(self, text: str):
        """ preprocess """
        tokens, valid_positions = self.tokenize(text)
        ## insert "[CLS]"
        tokens.insert(0,"[CLS]")
        valid_positions.insert(0,1)
        ## insert "[SEP]"
        tokens.append("[SEP]")
        valid_positions.append(1)
        segment_ids = []
        for i in range(len(tokens)):
            segment_ids.append(0)
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            valid_positions.append(0)
        return input_ids,input_mask,segment_ids,valid_positions

    def predict(self, text: str):
        input_ids,input_mask,segment_ids,valid_ids = self.preprocess(text)
        input_ids = torch.tensor([input_ids],dtype=torch.long,device=self.device)
        input_mask = torch.tensor([input_mask],dtype=torch.long,device=self.device)
        segment_ids = torch.tensor([segment_ids],dtype=torch.long,device=self.device)
        valid_ids = torch.tensor([valid_ids],dtype=torch.long,device=self.device)
        with torch.no_grad():
            logits = self.model(input_ids, segment_ids, input_mask,valid_ids)
        logits = F.softmax(logits,dim=2)
        logits_label = torch.argmax(logits,dim=2)
        logits_label = logits_label.detach().cpu().numpy().tolist()[0]

        logits_confidence = [values[label].item() for values,label in zip(logits[0],logits_label)]

        logits = []
        pos = 0
        for index,mask in enumerate(valid_ids[0]):
            if index == 0:
                continue
            if mask == 1:
                logits.append((logits_label[index-pos],logits_confidence[index-pos]))
            else:
                pos += 1
        logits.pop()

        labels = [(self.label_map[label],confidence) for label,confidence in logits]
        words = word_tokenize(text)
        assert len(labels) == len(words)

        Person = []
        Location = []
        Organization = []
        Miscelleneous = []

        for word, (label, confidence) in zip(words, labels):
            if label=="B-PER" or label=="I-PER":
                Person.append(word)
            elif label=="B-LOC" or label=="I-LOC":
                Location.append(word)
            elif label=="B-ORG" or label=="I-ORG":
                Organization.append(word)
            elif label=="B-MISC" or label=="I-MISC":
                Miscelleneous.append(word)
            else:
                output = None

        output = []
        for word, (label, confidence) in zip(words, labels):      
            if label == "B-PER":
                output.append(' '.join(Person) + ": Person")
            if label=="B-LOC":
                output.append(' '.join(Location) + ": Location")
            if label=="B-MISC":
                output.append(' '.join(Miscelleneous) + ": Miscelleneous Entity")
            if label=="B-ORG":
                output.append(' '.join(Organization) + ": Organization")
                
        return output



Put all the keyword lists into one list in order for BERT to make predictions on it. 

In [ ]:
keyword_lists = [keyword_list1_20, keyword_list2_20, keyword_list1_50, keyword_list2_50, keyword_list1_100,
                 keyword_list2_100, keyword_list1_200, keyword_list2_200, keyword_list1_500, keyword_list2_500,
                 keyword_list1_750, keyword_list2_750, keyword_list1_1000, keyword_list2_1000]

Next is some code to replace the first letter of a word with a captical letter, since NER gets a better performance with capital first letters for words referring to a person or organizations. Besides that the keywords are also transformed to input that BERT can take. 

I will change it all to beautiful for loops for the second submital. 

In [ ]:
# Python program to convert a list
# to string using join() function
    
# Function to convert  
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [ ]:
k_list1_20 = listToString(keyword_list1_20)
k_list2_20 = listToString(keyword_list2_20)
k_list1_50 = listToString(keyword_list1_50)
k_list2_50 = listToString(keyword_list2_50)
k_list1_100 = listToString(keyword_list1_100)
k_list2_100 = listToString(keyword_list2_100)
k_list1_200 = listToString(keyword_list1_200)
k_list2_200 = listToString(keyword_list2_200)
k_list1_500 = listToString(keyword_list1_500)
k_list2_500 = listToString(keyword_list2_500)
k_list1_750 = listToString(keyword_list1_750)
k_list2_750 = listToString(keyword_list2_750)
k_list1_1000 = listToString(keyword_list1_1000)
k_list2_1000 = listToString(keyword_list2_1000)

In [ ]:
k_list1_20 = k_list1_20.title().split()
k_list2_20 = k_list2_20.title().split()
k_list1_50 = k_list1_50.title().split()
k_list2_50 = k_list2_50.title().split()
k_list1_100 = k_list1_100.title().split()
k_list2_100 = k_list2_100.title().split()
k_list1_200 = k_list1_200.title().split()
k_list2_200 = k_list2_200.title().split()
k_list1_500 = k_list1_500.title().split()
k_list2_500 = k_list2_500.title().split()
k_list1_750 = k_list1_750.title().split()
k_list2_750 = k_list2_750.title().split()
k_list1_1000 = k_list1_1000.title().split()
k_list2_1000 = k_list2_1000.title().split()

Run the below cell for final output: 

In the below cell first line we call the '**Ner**' class from the 'bert.py' file. '**Ner**' class intialize our fine-tuned model.

Store the model in any variable. In below cell we store our fine-tuned model into 'model' variable.

Pass any text as a string for entity detection. 

Use '**predict**' function of class '**Ner**' for detecting entities for the 'text' and stored it into 'output'. In 'output' variable we have a detected entities of list.

Run 'for loop' for list formed 'output'. Print the 'prediction' of 'for loop'.

In [ ]:
from bert import Ner
model = Ner("out_ner/")

In [ ]:
new_k_list1_20 = []
for t in k_list1_20:
  output = model.predict(t)
  for p in output:
    new_k_list1_20.append(p)

In [ ]:
new_k_list2_20 = []
for t in k_list2_20:
  output = model.predict(t)
  for p in output:
    new_k_list2_20.append(p)

In [ ]:
new_k_list1_50 = []
for t in k_list1_50:
  output = model.predict(t)
  for p in output:
    new_k_list1_50.append(p)

In [ ]:
new_k_list2_50 = []
for t in k_list2_50:
  output = model.predict(t)
  for p in output:
    new_k_list2_50.append(p)

In [ ]:
new_k_list1_100 = []
for t in k_list1_100:
  output = model.predict(t)
  for p in output:
    new_k_list1_100.append(p)

In [ ]:
new_k_list2_100 = []
for t in k_list2_100:
  output = model.predict(t)
  for p in output:
    new_k_list2_100.append(p)

In [ ]:
new_k_list1_200 = []
for t in k_list1_200:
  output = model.predict(t)
  for p in output:
    new_k_list1_200.append(p)

In [ ]:
new_k_list2_200 = []
for t in k_list2_200:
  output = model.predict(t)
  for p in output:
    new_k_list2_200.append(p)

In [ ]:
new_k_list1_500 = []
for t in k_list1_500:
  output = model.predict(t)
  for p in output:
    new_k_list1_500.append(p)

In [ ]:
new_k_list2_500 = []
for t in k_list2_500:
  output = model.predict(t)
  for p in output:
    new_k_list2_500.append(p)

In [ ]:
new_k_list1_750 = []
for t in k_list1_750:
  output = model.predict(t)
  for p in output:
    new_k_list1_750.append(p)

In [ ]:
new_k_list2_750 = []
for t in k_list2_750:
  output = model.predict(t)
  for p in output:
    new_k_list2_750.append(p)

In [ ]:
new_k_list1_1000 = []
for t in k_list1_1000:
  output = model.predict(t)
  for p in output:
    new_k_list1_1000.append(p)

In [ ]:
new_k_list2_1000 = []
for t in k_list2_1000:
  output = model.predict(t)
  for p in output:
    new_k_list2_1000.append(p)

The NER-tags are now extracted on the keyword lists. Only the person and organization tags will be used in the current research. The following code makes sure the NER-keyword lists have the same format again as the regular keyword lists and the person-and organization NER tags are merged into one list. 

In [ ]:
person_keywords_1_20 = []
organization_keywords_1_20 = []

for match in new_k_list1_20:
    if "Person" in match:
      person_keywords_1_20.append(match)
    if "Organization" in match:
      organization_keywords_1_20.append(match)

print(person_keywords_1_20)
print(organization_keywords_1_20)

In [ ]:
person_keywords_2_20 = []
organization_keywords_2_20 = []

for match in new_k_list2_20:
    if "Person" in match:
      person_keywords_2_20.append(match)
    if "Organization" in match:
      organization_keywords_2_20.append(match)

print(person_keywords_2_20)
print(organization_keywords_2_20)

In [ ]:
person_keywords_1_50 = []
organization_keywords_1_50 = []

for match in new_k_list1_50:
    if "Person" in match:
      person_keywords_1_50.append(match)
    if "Organization" in match:
      organization_keywords_1_50.append(match)

print(person_keywords_1_50)
print(organization_keywords_1_50)

In [ ]:
person_keywords_2_50 = []
organization_keywords_2_50 = []

for match in new_k_list2_50:
    if "Person" in match:
      person_keywords_2_50.append(match)
    if "Organization" in match:
      organization_keywords_2_50.append(match)

print(person_keywords_2_50)
print(organization_keywords_2_50)

In [ ]:
person_keywords_1_100 = []
organization_keywords_1_100 = []

for match in new_k_list1_100:
    if "Person" in match:
      person_keywords_1_100.append(match)
    if "Organization" in match:
      organization_keywords_1_100.append(match)

print(person_keywords_1_100)
print(organization_keywords_1_100)

In [ ]:
person_keywords_2_100 = []
organization_keywords_2_100 = []

for match in new_k_list2_100:
    if "Person" in match:
      person_keywords_2_100.append(match)
    if "Organization" in match:
      organization_keywords_2_100.append(match)

print(person_keywords_2_100)
print(organization_keywords_2_100)

In [ ]:
person_keywords_1_200 = []
organization_keywords_1_200 = []

for match in new_k_list1_200:
    if "Person" in match:
      person_keywords_1_200.append(match)
    if "Organization" in match:
      organization_keywords_1_200.append(match)

print(person_keywords_1_200)
print(organization_keywords_1_200)

In [ ]:
person_keywords_2_200 = []
organization_keywords_2_200 = []

for match in new_k_list2_200:
    if "Person" in match:
      person_keywords_2_200.append(match)
    if "Organization" in match:
      organization_keywords_2_200.append(match)

print(person_keywords_2_200)
print(organization_keywords_2_200)

In [ ]:
person_keywords_1_500 = []
organization_keywords_1_500 = []

for match in new_k_list1_500:
    if "Person" in match:
      person_keywords_1_500.append(match)
    if "Organization" in match:
      organization_keywords_1_500.append(match)

print(person_keywords_1_500)
print(organization_keywords_1_500)

In [ ]:
person_keywords_2_500 = []
organization_keywords_2_500 = []

for match in new_k_list2_500:
    if "Person" in match:
      person_keywords_2_500.append(match)
    if "Organization" in match:
      organization_keywords_2_500.append(match)

print(person_keywords_2_500)
print(organization_keywords_2_500)

In [ ]:
person_keywords_1_750 = []
organization_keywords_1_750 = []

for match in new_k_list1_750:
    if "Person" in match:
      person_keywords_1_750.append(match)
    if "Organization" in match:
      organization_keywords_1_750.append(match)

print(person_keywords_1_750)
print(organization_keywords_1_750)

In [ ]:
person_keywords_2_750 = []
organization_keywords_2_750 = []

for match in new_k_list2_750:
    if "Person" in match:
      person_keywords_2_750.append(match)
    if "Organization" in match:
      organization_keywords_2_750.append(match)

print(person_keywords_2_750)
print(organization_keywords_2_750)

In [ ]:
person_keywords_1_1000 = []
organization_keywords_1_1000 = []

for match in new_k_list1_1000:
    if "Person" in match:
      person_keywords_1_1000.append(match)
    if "Organization" in match:
      organization_keywords_1_1000.append(match)

print(person_keywords_1_1000)
print(organization_keywords_1_1000)

In [ ]:
person_keywords_2_1000 = []
organization_keywords_2_1000 = []

for match in new_k_list2_1000:
    if "Person" in match:
      person_keywords_2_1000.append(match)
    if "Organization" in match:
      organization_keywords_2_1000.append(match)

print(person_keywords_2_1000)
print(organization_keywords_2_1000)

In [ ]:
person_1_20 = []
for p in person_keywords_1_20:
  p = str(p)
  person_1_20.append(p[:p.index(":")])

organization_1_20 = []
for o in organization_keywords_1_20:
  o = str(o)
  organization_1_20.append(o[:o.index(":")])

In [ ]:
person_2_20 = []
for p in person_keywords_2_20:
  p = str(p)
  person_2_20.append(p[:p.index(":")])

organization_2_20 = []
for o in organization_keywords_2_20:
  o = str(o)
  organization_2_20.append(o[:o.index(":")])

In [ ]:
person_1_50 = []
for p in person_keywords_1_50:
  p = str(p)
  person_1_50.append(p[:p.index(":")])

organization_1_50 = []
for o in organization_keywords_1_50:
  o = str(o)
  organization_1_50.append(o[:o.index(":")])

In [ ]:
person_2_50 = []
for p in person_keywords_2_50:
  p = str(p)
  person_2_50.append(p[:p.index(":")])

organization_2_50 = []
for o in organization_keywords_2_50:
  o = str(o)
  organization_2_50.append(o[:o.index(":")])

In [ ]:
person_1_100 = []
for p in person_keywords_1_100:
  p = str(p)
  person_1_100.append(p[:p.index(":")])

organization_1_100 = []
for o in organization_keywords_1_100:
  o = str(o)
  organization_1_100.append(o[:o.index(":")])

In [ ]:
person_2_100 = []
for p in person_keywords_2_100:
  p = str(p)
  person_2_100.append(p[:p.index(":")])

organization_2_100 = []
for o in organization_keywords_2_100:
  o = str(o)
  organization_2_100.append(o[:o.index(":")])

In [ ]:
person_1_200 = []
for p in person_keywords_1_200:
  p = str(p)
  person_1_200.append(p[:p.index(":")])

organization_1_200 = []
for o in organization_keywords_1_200:
  o = str(o)
  organization_1_200.append(o[:o.index(":")])

In [ ]:
person_2_200 = []
for p in person_keywords_2_200:
  p = str(p)
  person_2_200.append(p[:p.index(":")])

organization_2_200 = []
for o in organization_keywords_2_200:
  o = str(o)
  organization_2_200.append(o[:o.index(":")])

In [ ]:
person_1_500 = []
for p in person_keywords_1_500:
  p = str(p)
  person_1_500.append(p[:p.index(":")])

organization_1_500 = []
for o in organization_keywords_1_500:
  o = str(o)
  organization_1_500.append(o[:o.index(":")])

In [ ]:
person_2_500 = []
for p in person_keywords_2_500:
  p = str(p)
  person_2_500.append(p[:p.index(":")])

organization_2_500 = []
for o in organization_keywords_2_500:
  o = str(o)
  organization_2_500.append(o[:o.index(":")])

In [ ]:
person_1_750 = []
for p in person_keywords_1_750:
  p = str(p)
  person_1_750.append(p[:p.index(":")])

organization_1_750 = []
for o in organization_keywords_1_750:
  o = str(o)
  organization_1_750.append(o[:o.index(":")])

In [ ]:
person_2_750 = []
for p in person_keywords_2_750:
  p = str(p)
  person_2_750.append(p[:p.index(":")])

organization_2_750 = []
for o in organization_keywords_2_750:
  o = str(o)
  organization_2_750.append(o[:o.index(":")])

In [ ]:
person_1_1000 = []
for p in person_keywords_1_1000:
  p = str(p)
  person_1_1000.append(p[:p.index(":")])

organization_1_1000 = []
for o in organization_keywords_1_1000:
  o = str(o)
  organization_1_1000.append(o[:o.index(":")])

In [ ]:
person_2_1000 = []
for p in person_keywords_2_1000:
  p = str(p)
  person_2_1000.append(p[:p.index(":")])

organization_2_1000 = []
for o in organization_keywords_2_1000:
  o = str(o)
  organization_2_1000.append(o[:o.index(":")])

The following code is to save the NER-keyword lists. 

In [ ]:
ner_key_list1_20 = person_1_20 + organization_1_20

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list1_20.pickle', 'wb') as f:
  pickle.dump(ner_key_list1_20, f)

In [ ]:
ner_key_list2_20 = person_2_20 + organization_2_20

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list2_20.pickle', 'wb') as f:
  pickle.dump(ner_key_list2_20, f)

In [ ]:
ner_key_list1_50 = person_1_50 + organization_1_50

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list1_50.pickle', 'wb') as f:
  pickle.dump(ner_key_list1_50, f)

In [ ]:
ner_key_list2_50 = person_2_50 + organization_2_50

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list2_50.pickle', 'wb') as f:
  pickle.dump(ner_key_list2_50, f)

In [ ]:
ner_key_list1_100 = person_1_100 + organization_1_100

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list1_100.pickle', 'wb') as f:
  pickle.dump(ner_key_list1_100, f)

In [ ]:
ner_key_list2_100 = person_2_100 + organization_2_100

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list2_100.pickle', 'wb') as f:
  pickle.dump(ner_key_list2_100, f)

In [ ]:
ner_key_list1_200 = person_1_200 + organization_1_200

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list1_200.pickle', 'wb') as f:
  pickle.dump(ner_key_list1_200, f)

In [ ]:
ner_key_list2_200 = person_2_200 + organization_2_200

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list2_200.pickle', 'wb') as f:
  pickle.dump(ner_key_list2_200, f)

In [ ]:
ner_key_list1_500 = person_1_500 + organization_1_500

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list1_500.pickle', 'wb') as f:
  pickle.dump(ner_key_list1_500, f)

In [ ]:
ner_key_list2_500 = person_2_500 + organization_2_500

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list2_500.pickle', 'wb') as f:
  pickle.dump(ner_key_list2_500, f)

In [ ]:
ner_key_list1_750 = person_1_750 + organization_1_750

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list1_750.pickle', 'wb') as f:
  pickle.dump(ner_key_list1_750, f)

In [ ]:
ner_key_list2_750 = person_2_750 + organization_2_750

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list2_750.pickle', 'wb') as f:
  pickle.dump(ner_key_list2_750, f)

In [ ]:
ner_key_list1_1000 = person_1_1000 + organization_1_1000

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list1_1000.pickle', 'wb') as f:
  pickle.dump(ner_key_list1_1000, f)

In [ ]:
ner_key_list2_1000 = person_2_1000 + organization_2_1000

os.chdir("/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003")
with open('ner_key_list2_1000.pickle', 'wb') as f:
  pickle.dump(ner_key_list2_1000, f)

Once the NER keyword lists are created and saved, the following code can be used to load it. The first letter of each word is returned to a lower letter again, to ensure the keyword lists and NER-keyword lists have the same format.   

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003'
os.chdir(filepath)
import pickle

keyword = 20
with open('ner_key_list1_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list1_20 = pickle.load(f)
with open('ner_key_list2_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list2_20 = pickle.load(f)

ner_keyword_list1_20 = [x.lower() for x in ner_keyword_list1_20]
ner_keyword_list2_20 = [x.lower() for x in ner_keyword_list2_20]

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003'
os.chdir(filepath)
import pickle

keyword = 50
with open('ner_key_list1_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list1_50 = pickle.load(f)
with open('ner_key_list2_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list2_50 = pickle.load(f)
  
ner_keyword_list1_50 = [x.lower() for x in ner_keyword_list1_50]
ner_keyword_list2_50 = [x.lower() for x in ner_keyword_list2_50]

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003'
os.chdir(filepath)
import pickle

keyword = 100
with open('ner_key_list1_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list1_100 = pickle.load(f)
with open('ner_key_list2_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list2_100 = pickle.load(f)

ner_keyword_list1_100 = [x.lower() for x in ner_keyword_list1_100]
ner_keyword_list2_100 = [x.lower() for x in ner_keyword_list2_100]

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003'
os.chdir(filepath)
import pickle

keyword = 200
with open('ner_key_list1_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list1_200 = pickle.load(f)
with open('ner_key_list2_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list2_200 = pickle.load(f)

ner_keyword_list1_200 = [x.lower() for x in ner_keyword_list1_200]
ner_keyword_list2_200 = [x.lower() for x in ner_keyword_list2_200]

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003'
os.chdir(filepath)
import pickle

keyword = 500
with open('ner_key_list1_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list1_500 = pickle.load(f)
with open('ner_key_list2_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list2_500 = pickle.load(f)

ner_keyword_list1_500 = [x.lower() for x in ner_keyword_list1_500]
ner_keyword_list2_500 = [x.lower() for x in ner_keyword_list2_500]

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003'
os.chdir(filepath)
import pickle

keyword = 750
with open('ner_key_list1_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list1_750 = pickle.load(f)
with open('ner_key_list2_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list2_750 = pickle.load(f)

ner_keyword_list1_750 = [x.lower() for x in ner_keyword_list1_750]
ner_keyword_list2_750 = [x.lower() for x in ner_keyword_list2_750]

In [ ]:
import os
filepath = '/content/drive/My Drive/Colab Notebooks/Thesis Project/Data exploration/Keywords 1 NER CONLL2003'
os.chdir(filepath)
import pickle

keyword = 1000
with open('ner_key_list1_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list1_1000 = pickle.load(f)
with open('ner_key_list2_' + str(keyword) + '.pickle', 'rb') as f:
    ner_keyword_list2_1000 = pickle.load(f)

ner_keyword_list1_1000 = [x.lower() for x in ner_keyword_list1_1000]
ner_keyword_list2_1000 = [x.lower() for x in ner_keyword_list2_1000]

Transformation of the training and test data

In [ ]:
#for training I fit and transform the train data:
tv_training = TfidfVectorizer(stop_words=None, max_features=4000)
tf_idf_prel_training = tv_training.fit_transform(tfidf_input)
tf_idf_vector_training = tf_idf_prel_training.toarray()
tf_idf_feature_names_training = tv_training.get_feature_names()

#for test, using the same vectorizer I only transform the test data
tf_idf_prel_test = tv_training.transform(tfidf_input_test)
tf_idf_vector_test = tf_idf_prel_test.toarray()

In [ ]:
### We route this back in X and Y variables (X_val & y_val for the validation stage, X_ts0 & y_tst for the final test set)
#
X_val = tf_idf_vector_training
y_val = []

In [ ]:
# Here we condense class 2 & 3 into one class, leaving is with 0 (non conspiratorial) or 1 (conspiratorial)
for document in y_trainingdata:
    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val.append(class_made)

In [ ]:
import numpy as np
y_val = np.array(y_val)

In [ ]:
X_tst = tf_idf_vector_test
y_tst = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst.append(class_made)

y_tst = np.array(y_tst)

**BIAS 1 - BASE**

In [ ]:
# we need gensims word2vec here
import gensim
import logging

**Keyword = 20**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1_20 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_20:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1_20.append(counter)

# I now have a matrix containing a multiplier score for each word in the TF-IDF Vocabulary, based on similarity to our keyword lists
# now I multiply that matrix with each row of our TF-IDF vectors in both training and test sets
X_val_biased_1_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias1_20[j]*item)

    X_val_biased_1_20.append(temp_num_20)
X_val_biased_1_20 = np.array(X_val_biased_1_20)

X_test_biased_1_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias1_20[j]*item)

    X_test_biased_1_20.append(temp_num_20)
X_test_biased_1_20 = np.array(X_test_biased_1_20)

# Now I route this back in X and Y variables
#for training:
X_val_bias1_20 = X_val_biased_1_20
y_val_bias1_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1_20.append(class_made)

y_val_bias1_20 = np.array(y_val_bias1_20)

# for testing
X_tst_bias1_20 = X_test_biased_1_20
y_tst_bias1_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1_20.append(class_made)

y_tst_bias1_20 = np.array(y_tst_bias1_20)

**Keyword = 50**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1_50 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_50:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1_50.append(counter)

X_val_biased_1_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias1_50[j]*item)

    X_val_biased_1_50.append(temp_num_50)
X_val_biased_1_50 = np.array(X_val_biased_1_50)

X_test_biased_1_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias1_50[j]*item)

    X_test_biased_1_50.append(temp_num_50)
X_test_biased_1_50 = np.array(X_test_biased_1_50)

X_val_bias1_50 = X_val_biased_1_50
y_val_bias1_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1_50.append(class_made)

y_val_bias1_50 = np.array(y_val_bias1_50)

# for testing
X_tst_bias1_50 = X_test_biased_1_50
y_tst_bias1_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1_50.append(class_made)

y_tst_bias1_50 = np.array(y_tst_bias1_50)

**Keyword = 100**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1_100 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_100:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1_100.append(counter)

X_val_biased_1_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias1_100[j]*item)

    X_val_biased_1_100.append(temp_num_100)
X_val_biased_1_100 = np.array(X_val_biased_1_100)

X_test_biased_1_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias1_100[j]*item)

    X_test_biased_1_100.append(temp_num_100)
X_test_biased_1_100 = np.array(X_test_biased_1_100)

X_val_bias1_100 = X_val_biased_1_100
y_val_bias1_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1_100.append(class_made)

y_val_bias1_100 = np.array(y_val_bias1_100)

# for testing
X_tst_bias1_100 = X_test_biased_1_100
y_tst_bias1_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1_100.append(class_made)

y_tst_bias1_100 = np.array(y_tst_bias1_100)

**Keyword = 200**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1_200 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_200:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1_200.append(counter)

X_val_biased_1_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias1_200[j]*item)

    X_val_biased_1_200.append(temp_num_200)
X_val_biased_1_200 = np.array(X_val_biased_1_200)

X_test_biased_1_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias1_200[j]*item)

    X_test_biased_1_200.append(temp_num_200)
X_test_biased_1_200 = np.array(X_test_biased_1_200)

X_val_bias1_200 = X_val_biased_1_200
y_val_bias1_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1_200.append(class_made)

y_val_bias1_200 = np.array(y_val_bias1_200)

# for testing
X_tst_bias1_200 = X_test_biased_1_200
y_tst_bias1_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1_200.append(class_made)

y_tst_bias1_200 = np.array(y_tst_bias1_200)

**Keyword = 500**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1_500 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_500:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1_500.append(counter)

X_val_biased_1_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias1_500[j]*item)

    X_val_biased_1_500.append(temp_num_500)
X_val_biased_1_500 = np.array(X_val_biased_1_500)

X_test_biased_1_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias1_500[j]*item)

    X_test_biased_1_500.append(temp_num_500)
X_test_biased_1_500 = np.array(X_test_biased_1_500)

X_val_bias1_500 = X_val_biased_1_500
y_val_bias1_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1_500.append(class_made)

y_val_bias1_500 = np.array(y_val_bias1_500)

# for testing
X_tst_bias1_500 = X_test_biased_1_500
y_tst_bias1_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1_500.append(class_made)

y_tst_bias1_500 = np.array(y_tst_bias1_500)

**Keyword = 750**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1_750 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_750:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1_750.append(counter)
  
X_val_biased_1_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias1_750[j]*item)

    X_val_biased_1_750.append(temp_num_750)
X_val_biased_1_750 = np.array(X_val_biased_1_750)

X_test_biased_1_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias1_750[j]*item)

    X_test_biased_1_750.append(temp_num_750)
X_test_biased_1_750 = np.array(X_test_biased_1_750)

X_val_bias1_750 = X_val_biased_1_750
y_val_bias1_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1_750.append(class_made)

y_val_bias1_750 = np.array(y_val_bias1_750)

# for testing
X_tst_bias1_750 = X_test_biased_1_750
y_tst_bias1_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1_750.append(class_made)

y_tst_bias1_750 = np.array(y_tst_bias1_750)

**Keyword = 1000**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1_1000 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_1000:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1_1000.append(counter)

X_val_biased_1_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias1_1000[j]*item)

    X_val_biased_1_1000.append(temp_num_1000)
X_val_biased_1_1000 = np.array(X_val_biased_1_1000)

X_test_biased_1_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias1_1000[j]*item)

    X_test_biased_1_1000.append(temp_num_1000)
X_test_biased_1_1000 = np.array(X_test_biased_1_1000)

X_val_bias1_1000 = X_val_biased_1_1000
y_val_bias1_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1_1000.append(class_made)

y_val_bias1_1000 = np.array(y_val_bias1_1000)

# for testing
X_tst_bias1_1000 = X_test_biased_1_1000
y_tst_bias1_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1_1000.append(class_made)

y_tst_bias1_1000 = np.array(y_tst_bias1_1000)

## **SVM BIAS 1 - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validation = X_val_bias1_20, X_val_bias1_50, X_val_bias1_100, X_val_bias1_200, X_val_bias1_500, X_val_bias1_750, X_val_bias1_1000
y_validation = y_val_bias1_20, y_val_bias1_50, y_val_bias1_100, y_val_bias1_200, y_val_bias1_500, y_val_bias1_750, y_val_bias1_1000

X_tst_bias1 = X_tst_bias1_20, X_tst_bias1_50, X_tst_bias1_100, X_tst_bias1_200, X_tst_bias1_500, X_tst_bias1_750, X_tst_bias1_1000
y_tst_bias1 = y_tst_bias1_20, y_tst_bias1_50, y_tst_bias1_100, y_tst_bias1_200, y_tst_bias1_500, y_tst_bias1_750, y_tst_bias1_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validation, y_validation, X_tst_bias1, y_tst_bias1):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier2 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier2.fit(X_train, y_train)

# validation
    print("SVM validation bias1", "SVM_C = ", c, "number = ", count)
    y_pred2 = classifier2.predict(X_test)
    print(confusion_matrix(y_test, y_pred2))
    print()
    print(classification_report(y_test, y_pred2))
    print()
    print(precision_recall_fscore_support(y_test, y_pred2, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred2, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred2))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias1", "SVM_C = ", c, "number = ", count)
    y_pred2_test = classifier2.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred2_test))
    print()
    print(classification_report(a, y_pred2_test))
    print()
    print(precision_recall_fscore_support(a, y_pred2_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred2_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred2_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred2_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")

**BIAS 2 - BASE**

**Keyword = 20**

In [ ]:
# now to bias the X for bias 2
X_bias2_20 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_20:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2_20.append(counter)

X_val_biased_2_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias2_20[j]*item)

    X_val_biased_2_20.append(temp_num_20)
X_val_biased_2_20 = np.array(X_val_biased_2_20)

X_test_biased_2_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias2_20[j]*item)

    X_test_biased_2_20.append(temp_num_20)
X_test_biased_2_20 = np.array(X_test_biased_2_20)

X_val_bias2_20 = X_val_biased_2_20
y_val_bias2_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2_20.append(class_made)

y_val_bias2_20 = np.array(y_val_bias2_20)

X_tst_bias2_20 = X_test_biased_2_20
y_tst_bias2_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2_20.append(class_made)

y_tst_bias2_20 = np.array(y_tst_bias2_20)

**Keyword = 50**

In [ ]:
# now to bias the X for bias 2
X_bias2_50 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_50:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2_50.append(counter)

X_val_biased_2_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias2_50[j]*item)

    X_val_biased_2_50.append(temp_num_50)
X_val_biased_2_50 = np.array(X_val_biased_2_50)

X_test_biased_2_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias2_50[j]*item)

    X_test_biased_2_50.append(temp_num_50)
X_test_biased_2_50 = np.array(X_test_biased_2_50)

X_val_bias2_50 = X_val_biased_2_50
y_val_bias2_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2_50.append(class_made)

y_val_bias2_50 = np.array(y_val_bias2_50)

X_tst_bias2_50 = X_test_biased_2_50
y_tst_bias2_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2_50.append(class_made)

y_tst_bias2_50 = np.array(y_tst_bias2_50)

**Keyword = 100**

In [ ]:
# now to bias the X for bias 2
X_bias2_100 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_100:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2_100.append(counter)

X_val_biased_2_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias2_100[j]*item)

    X_val_biased_2_100.append(temp_num_100)
X_val_biased_2_100 = np.array(X_val_biased_2_100)

X_test_biased_2_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias2_100[j]*item)

    X_test_biased_2_100.append(temp_num_100)
X_test_biased_2_100 = np.array(X_test_biased_2_100)

X_val_bias2_100 = X_val_biased_2_100
y_val_bias2_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2_100.append(class_made)

y_val_bias2_100 = np.array(y_val_bias2_100)

X_tst_bias2_100 = X_test_biased_2_100
y_tst_bias2_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2_100.append(class_made)

y_tst_bias2_100 = np.array(y_tst_bias2_100)

**Keyword = 200**

In [ ]:
# now to bias the X for bias 2
X_bias2_200 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_200:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2_200.append(counter)

X_val_biased_2_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias2_200[j]*item)

    X_val_biased_2_200.append(temp_num_200)
X_val_biased_2_200 = np.array(X_val_biased_2_200)

X_test_biased_2_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias2_200[j]*item)

    X_test_biased_2_200.append(temp_num_200)
X_test_biased_2_200 = np.array(X_test_biased_2_200)

X_val_bias2_200 = X_val_biased_2_200
y_val_bias2_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2_200.append(class_made)

y_val_bias2_200 = np.array(y_val_bias2_200)

X_tst_bias2_200 = X_test_biased_2_200
y_tst_bias2_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2_200.append(class_made)

y_tst_bias2_200 = np.array(y_tst_bias2_200)

**Keyword = 500**

In [ ]:
# now to bias the X for bias 2
X_bias2_500 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_500:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2_500.append(counter)

X_val_biased_2_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias2_500[j]*item)

    X_val_biased_2_500.append(temp_num_500)
X_val_biased_2_500 = np.array(X_val_biased_2_500)

X_test_biased_2_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias2_500[j]*item)

    X_test_biased_2_500.append(temp_num_500)
X_test_biased_2_500 = np.array(X_test_biased_2_500)

X_val_bias2_500 = X_val_biased_2_500
y_val_bias2_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2_500.append(class_made)

y_val_bias2_500 = np.array(y_val_bias2_500)

X_tst_bias2_500 = X_test_biased_2_500
y_tst_bias2_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2_500.append(class_made)

y_tst_bias2_500 = np.array(y_tst_bias2_500)

**Keyword = 750**

In [ ]:
# now to bias the X for bias 2
X_bias2_750 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_750:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2_750.append(counter)

X_val_biased_2_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias2_750[j]*item)

    X_val_biased_2_750.append(temp_num_750)
X_val_biased_2_750 = np.array(X_val_biased_2_750)

X_test_biased_2_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias2_750[j]*item)

    X_test_biased_2_750.append(temp_num_750)
X_test_biased_2_750 = np.array(X_test_biased_2_750)

X_val_bias2_750 = X_val_biased_2_750
y_val_bias2_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2_750.append(class_made)

y_val_bias2_750 = np.array(y_val_bias2_750)

X_tst_bias2_750 = X_test_biased_2_750
y_tst_bias2_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2_750.append(class_made)

y_tst_bias2_750 = np.array(y_tst_bias2_750)

**Keyword = 1000**

In [ ]:
# now to bias the X for bias 2
X_bias2_1000 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_1000:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2_1000.append(counter)

X_val_biased_2_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias2_1000[j]*item)

    X_val_biased_2_1000.append(temp_num_1000)
X_val_biased_2_1000 = np.array(X_val_biased_2_1000)

X_test_biased_2_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias2_1000[j]*item)

    X_test_biased_2_1000.append(temp_num_1000)
X_test_biased_2_1000 = np.array(X_test_biased_2_1000)

X_val_bias2_1000 = X_val_biased_2_1000
y_val_bias2_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2_1000.append(class_made)

y_val_bias2_1000 = np.array(y_val_bias2_1000)

X_tst_bias2_1000 = X_test_biased_2_1000
y_tst_bias2_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2_1000.append(class_made)

y_tst_bias2_1000 = np.array(y_tst_bias2_1000)

## **SVM BIAS 2 - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validation = X_val_bias2_20, X_val_bias2_50, X_val_bias2_100, X_val_bias2_200, X_val_bias2_500, X_val_bias2_750, X_val_bias2_1000
y_validation = y_val_bias2_20, y_val_bias2_50, y_val_bias2_100, y_val_bias2_200, y_val_bias2_500, y_val_bias2_750, y_val_bias2_1000

X_tst_bias2 = X_tst_bias2_20, X_tst_bias2_50, X_tst_bias2_100, X_tst_bias2_200, X_tst_bias2_500, X_tst_bias2_750, X_tst_bias2_1000
y_tst_bias2 = y_tst_bias2_20, y_tst_bias2_50, y_tst_bias2_100, y_tst_bias2_200, y_tst_bias2_500, y_tst_bias2_750, y_tst_bias2_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validation, y_validation, X_tst_bias2, y_tst_bias2):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier3 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier3.fit(X_train, y_train)

# validation
    print("SVM validation bias2", "SVM_C = ", c, "number = ", count)
    y_pred3 = classifier3.predict(X_test)
    print(confusion_matrix(y_test, y_pred3))
    print()
    print(classification_report(y_test, y_pred3))
    print()
    print(precision_recall_fscore_support(y_test, y_pred3, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred3, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred3))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias2", "SVM_C = ", c, "number = ", count)
    y_pred3_test = classifier3.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred3_test))
    print()
    print(classification_report(a, y_pred3_test))
    print()
    print(precision_recall_fscore_support(a, y_pred3_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred3_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred3_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred3_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")

**BIAS 3 - BASE**

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# load the pretrained file GLove for word similairty
glove_file = datapath('/content/drive/My Drive/Colab Notebooks/Thesis Project/Datasets GloVe/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile('glove.6B.100d.txt')

glove2word2vec(glove_file,word2vec_glove_file)

**Keyword = 20**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3_20 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_20:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3_20.append(counter)

X_val_biased_3_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias3_20[j]*item)

    X_val_biased_3_20.append(temp_num_20)
X_val_biased_3_20 = np.array(X_val_biased_3_20)

X_test_biased_3_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias3_20[j]*item)

    X_test_biased_3_20.append(temp_num_20)
X_test_biased_3_20 = np.array(X_test_biased_3_20)

X_val_bias3_20 = X_val_biased_3_20
y_val_bias3_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3_20.append(class_made)

y_val_bias3_20 = np.array(y_val_bias3_20)

X_tst_bias3_20 = X_test_biased_3_20
y_tst_bias3_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3_20.append(class_made)

y_tst_bias3_20 = np.array(y_tst_bias3_20)

**Keyword = 50**

In [ ]:
# create biases in the same fashion as before
X_bias3_50 = []

# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_50:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3_50.append(counter)

X_val_biased_3_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias3_50[j]*item)

    X_val_biased_3_50.append(temp_num_50)
X_val_biased_3_50 = np.array(X_val_biased_3_50)

X_test_biased_3_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias3_50[j]*item)

    X_test_biased_3_50.append(temp_num_50)
X_test_biased_3_50 = np.array(X_test_biased_3_50)

X_val_bias3_50 = X_val_biased_3_50
y_val_bias3_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3_50.append(class_made)

y_val_bias3_50 = np.array(y_val_bias3_50)

X_tst_bias3_50 = X_test_biased_3_50
y_tst_bias3_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3_50.append(class_made)

y_tst_bias3_50 = np.array(y_tst_bias3_50)

**Keyword = 100**

In [ ]:
# create biases in the same fashion as before
X_bias3_100 = []

# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_100:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3_100.append(counter)

X_val_biased_3_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias3_100[j]*item)

    X_val_biased_3_100.append(temp_num_100)
X_val_biased_3_100 = np.array(X_val_biased_3_100)

X_test_biased_3_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias3_100[j]*item)

    X_test_biased_3_100.append(temp_num_100)
X_test_biased_3_100 = np.array(X_test_biased_3_100)

X_val_bias3_100 = X_val_biased_3_100
y_val_bias3_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3_100.append(class_made)

y_val_bias3_100 = np.array(y_val_bias3_100)

X_tst_bias3_100 = X_test_biased_3_100
y_tst_bias3_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3_100.append(class_made)

y_tst_bias3_100 = np.array(y_tst_bias3_100)

**Keyword = 200**

In [ ]:
# create biases in the same fashion as before
X_bias3_200 = []

# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_200:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3_200.append(counter)

X_val_biased_3_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias3_200[j]*item)

    X_val_biased_3_200.append(temp_num_200)
X_val_biased_3_200 = np.array(X_val_biased_3_200)

X_test_biased_3_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias3_200[j]*item)

    X_test_biased_3_200.append(temp_num_200)
X_test_biased_3_200 = np.array(X_test_biased_3_200)

X_val_bias3_200 = X_val_biased_3_200
y_val_bias3_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3_200.append(class_made)

y_val_bias3_200 = np.array(y_val_bias3_200)

X_tst_bias3_200 = X_test_biased_3_200
y_tst_bias3_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3_200.append(class_made)

y_tst_bias3_200 = np.array(y_tst_bias3_200)

**Keyword = 500**

In [ ]:
# create biases in the same fashion as before
X_bias3_500 = []

# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_500:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3_500.append(counter)

X_val_biased_3_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias3_500[j]*item)

    X_val_biased_3_500.append(temp_num_500)
X_val_biased_3_500 = np.array(X_val_biased_3_500)

X_test_biased_3_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias3_500[j]*item)

    X_test_biased_3_500.append(temp_num_500)
X_test_biased_3_500 = np.array(X_test_biased_3_500)

X_val_bias3_500 = X_val_biased_3_500
y_val_bias3_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3_500.append(class_made)

y_val_bias3_500 = np.array(y_val_bias3_500)

X_tst_bias3_500 = X_test_biased_3_500
y_tst_bias3_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3_500.append(class_made)

y_tst_bias3_500 = np.array(y_tst_bias3_500)

**Keyword = 750**

In [ ]:
# create biases in the same fashion as before
X_bias3_750 = []

# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_750:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3_750.append(counter)

X_val_biased_3_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias3_750[j]*item)

    X_val_biased_3_750.append(temp_num_750)
X_val_biased_3_750 = np.array(X_val_biased_3_750)

X_test_biased_3_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias3_750[j]*item)

    X_test_biased_3_750.append(temp_num_750)
X_test_biased_3_750 = np.array(X_test_biased_3_750)

X_val_bias3_750 = X_val_biased_3_750
y_val_bias3_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3_750.append(class_made)

y_val_bias3_750 = np.array(y_val_bias3_750)

X_tst_bias3_750 = X_test_biased_3_750
y_tst_bias3_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3_750.append(class_made)

y_tst_bias3_750 = np.array(y_tst_bias3_750)

**Keyword = 1000**

In [ ]:
# create biases in the same fashion as before
X_bias3_1000 = []

# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list1_1000:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3_1000.append(counter)

X_val_biased_3_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias3_1000[j]*item)

    X_val_biased_3_1000.append(temp_num_1000)
X_val_biased_3_1000 = np.array(X_val_biased_3_1000)

X_test_biased_3_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias3_1000[j]*item)

    X_test_biased_3_1000.append(temp_num_1000)
X_test_biased_3_1000 = np.array(X_test_biased_3_1000)

X_val_bias3_1000 = X_val_biased_3_1000
y_val_bias3_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3_1000.append(class_made)

y_val_bias3_1000 = np.array(y_val_bias3_1000)

X_tst_bias3_1000 = X_test_biased_3_1000
y_tst_bias3_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3_1000.append(class_made)

y_tst_bias3_1000 = np.array(y_tst_bias3_1000)

## **SVM BIAS 3 - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validation = X_val_bias3_20, X_val_bias3_50, X_val_bias3_100, X_val_bias3_200, X_val_bias3_500, X_val_bias3_750, X_val_bias3_1000
y_validation = y_val_bias3_20, y_val_bias3_50, y_val_bias3_100, y_val_bias3_200, y_val_bias3_500, y_val_bias3_750, y_val_bias3_1000

X_tst_bias3 = X_tst_bias3_20, X_tst_bias3_50, X_tst_bias3_100, X_tst_bias3_200, X_tst_bias3_500, X_tst_bias3_750, X_tst_bias3_1000
y_tst_bias3 = y_tst_bias3_20, y_tst_bias3_50, y_tst_bias3_100, y_tst_bias3_200, y_tst_bias3_500, y_tst_bias3_750, y_tst_bias3_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validation, y_validation, X_tst_bias3, y_tst_bias3):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier4 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier4.fit(X_train, y_train)

# validation
    print("SVM validation bias3", "SVM_C = ", c, "number = ", count)
    y_pred4 = classifier4.predict(X_test)
    print(confusion_matrix(y_test, y_pred4))
    print()
    print(classification_report(y_test, y_pred4))
    print()
    print(precision_recall_fscore_support(y_test, y_pred4, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred4, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred4))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias3", "SVM_C = ", c, "number = ", count)
    y_pred4_test = classifier4.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred4_test))
    print()
    print(classification_report(a, y_pred4_test))
    print()
    print(precision_recall_fscore_support(a, y_pred4_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred4_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred4_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred4_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")

**BIAS 4 - BASE**

**Keyword = 20**

In [ ]:
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# now for the last bias
X_bias4_20 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_20:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass


        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4_20.append(counter)

X_val_biased_4_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias4_20[j]*item)

    X_val_biased_4_20.append(temp_num_20)
X_val_biased_4_20 = np.array(X_val_biased_4_20)

X_test_biased_4_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias4_20[j]*item)

    X_test_biased_4_20.append(temp_num_20)
X_test_biased_4_20 = np.array(X_test_biased_4_20)

X_val_bias4_20 = X_val_biased_4_20
y_val_bias4_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4_20.append(class_made)

y_val_bias4_20 = np.array(y_val_bias4_20)

X_tst_bias4_20 = X_test_biased_4_20
y_tst_bias4_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4_20.append(class_made)

y_tst_bias4_20 = np.array(y_tst_bias4_20)

**Keyword = 50**

In [ ]:
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# now for the last bias
X_bias4_50 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_50:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass


        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4_50.append(counter)

X_val_biased_4_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias4_50[j]*item)

    X_val_biased_4_50.append(temp_num_50)
X_val_biased_4_50 = np.array(X_val_biased_4_50)

X_test_biased_4_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias4_50[j]*item)

    X_test_biased_4_50.append(temp_num_50)
X_test_biased_4_50 = np.array(X_test_biased_4_50)

X_val_bias4_50 = X_val_biased_4_50
y_val_bias4_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4_50.append(class_made)

y_val_bias4_50 = np.array(y_val_bias4_50)

X_tst_bias4_50 = X_test_biased_4_50
y_tst_bias4_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4_50.append(class_made)

y_tst_bias4_50 = np.array(y_tst_bias4_50)

**Keyword = 100**

In [ ]:
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# now for the last bias
X_bias4_100 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_100:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass


        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4_100.append(counter)

X_val_biased_4_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias4_100[j]*item)

    X_val_biased_4_100.append(temp_num_100)
X_val_biased_4_100 = np.array(X_val_biased_4_100)

X_test_biased_4_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias4_100[j]*item)

    X_test_biased_4_100.append(temp_num_100)
X_test_biased_4_100 = np.array(X_test_biased_4_100)

X_val_bias4_100 = X_val_biased_4_100
y_val_bias4_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4_100.append(class_made)

y_val_bias4_100 = np.array(y_val_bias4_100)

X_tst_bias4_100 = X_test_biased_4_100
y_tst_bias4_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4_100.append(class_made)

y_tst_bias4_100 = np.array(y_tst_bias4_100)

**Keyword = 200**

In [ ]:
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# now for the last bias
X_bias4_200 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_200:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass


        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4_200.append(counter)

X_val_biased_4_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias4_200[j]*item)

    X_val_biased_4_200.append(temp_num_200)
X_val_biased_4_200 = np.array(X_val_biased_4_200)

X_test_biased_4_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias4_200[j]*item)

    X_test_biased_4_200.append(temp_num_200)
X_test_biased_4_200 = np.array(X_test_biased_4_200)

X_val_bias4_200 = X_val_biased_4_200
y_val_bias4_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4_200.append(class_made)

y_val_bias4_200 = np.array(y_val_bias4_200)

X_tst_bias4_200 = X_test_biased_4_200
y_tst_bias4_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4_200.append(class_made)

y_tst_bias4_200 = np.array(y_tst_bias4_200)

**Keyword = 500**

In [ ]:
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# now for the last bias
X_bias4_500 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_500:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass


        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4_500.append(counter)

X_val_biased_4_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias4_500[j]*item)

    X_val_biased_4_500.append(temp_num_500)
X_val_biased_4_500 = np.array(X_val_biased_4_500)

X_test_biased_4_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias4_500[j]*item)

    X_test_biased_4_500.append(temp_num_500)
X_test_biased_4_500 = np.array(X_test_biased_4_500)

X_val_bias4_500 = X_val_biased_4_500
y_val_bias4_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4_500.append(class_made)

y_val_bias4_500 = np.array(y_val_bias4_500)

X_tst_bias4_500 = X_test_biased_4_500
y_tst_bias4_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4_500.append(class_made)

y_tst_bias4_500 = np.array(y_tst_bias4_500)

**Keyword = 750**

In [ ]:
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# now for the last bias
X_bias4_750 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_750:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass


        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4_750.append(counter)

X_val_biased_4_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias4_750[j]*item)

    X_val_biased_4_750.append(temp_num_750)
X_val_biased_4_750 = np.array(X_val_biased_4_750)

X_test_biased_4_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias4_750[j]*item)

    X_test_biased_4_750.append(temp_num_750)
X_test_biased_4_750 = np.array(X_test_biased_4_750)

X_val_bias4_750 = X_val_biased_4_750
y_val_bias4_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4_750.append(class_made)

y_val_bias4_750 = np.array(y_val_bias4_750)

X_tst_bias4_750 = X_test_biased_4_750
y_tst_bias4_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4_750.append(class_made)

y_tst_bias4_750 = np.array(y_tst_bias4_750)

**Keyword = 1000**

In [ ]:
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# now for the last bias
X_bias4_1000 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in keyword_list2_1000:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass


        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4_1000.append(counter)

X_val_biased_4_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias4_1000[j]*item)

    X_val_biased_4_1000.append(temp_num_1000)
X_val_biased_4_1000 = np.array(X_val_biased_4_1000)

X_test_biased_4_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias4_1000[j]*item)

    X_test_biased_4_1000.append(temp_num_1000)
X_test_biased_4_1000 = np.array(X_test_biased_4_1000)

X_val_bias4_1000 = X_val_biased_4_1000
y_val_bias4_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4_1000.append(class_made)

y_val_bias4_1000 = np.array(y_val_bias4_1000)

X_tst_bias4_1000 = X_test_biased_4_1000
y_tst_bias4_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4_1000.append(class_made)

y_tst_bias4_1000 = np.array(y_tst_bias4_1000)

## **SVM BIAS 4 - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validation = X_val_bias4_20, X_val_bias4_50, X_val_bias4_100, X_val_bias4_200, X_val_bias4_500, X_val_bias4_750, X_val_bias4_1000
y_validation = y_val_bias4_20, y_val_bias4_50, y_val_bias4_100, y_val_bias4_200, y_val_bias4_500, y_val_bias4_750, y_val_bias4_1000

X_tst_bias4 = X_tst_bias4_20, X_tst_bias4_50, X_tst_bias4_100, X_tst_bias4_200, X_tst_bias4_500, X_tst_bias4_750, X_tst_bias4_1000
y_tst_bias4 = y_tst_bias4_20, y_tst_bias4_50, y_tst_bias4_100, y_tst_bias4_200, y_tst_bias4_500, y_tst_bias4_750, y_tst_bias4_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validation, y_validation, X_tst_bias4, y_tst_bias4):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier6 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier6.fit(X_train, y_train)

# validation
    print("SVM validation bias4", "SVM_C = ", c, "number = ", count)
    y_pred5 = classifier6.predict(X_test)
    print(confusion_matrix(y_test, y_pred5))
    print()
    print(classification_report(y_test, y_pred5))
    print()
    print(precision_recall_fscore_support(y_test, y_pred5, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred5, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred5))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias4", "SVM_C = ", c, "number = ", count)
    y_pred5_test = classifier6.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred5_test))
    print()
    print(classification_report(a, y_pred5_test))
    print()
    print(precision_recall_fscore_support(a, y_pred5_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred5_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred5_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred5_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")

# NER BIASED CLASSIFIERS

**BIAS 1 - NER (CoNLL-2003)**

**Keyword = 20**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1c_20 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_20:
          try:
              counter += model.wv.similarity(w1, w2)
              averager += 1
          except:
              averager -= 1
              pass

        
        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1c_20.append(counter)

X_val_biased_1c_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias1c_20[j]*(item*0.8))

    X_val_biased_1c_20.append(temp_num_20)
X_val_biased_1c_20 = np.array(X_val_biased_1c_20)

X_test_biased_1c_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias1c_20[j]*(item*0.8))

    X_test_biased_1c_20.append(temp_num_20)
X_test_biased_1c_20 = np.array(X_test_biased_1c_20)

X_val_bias1c_20 = X_val_biased_1c_20
y_val_bias1c_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1c_20.append(class_made)

y_val_bias1c_20 = np.array(y_val_bias1c_20)

# for testing
X_tst_bias1c_20 = X_test_biased_1c_20
y_tst_bias1c_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1c_20.append(class_made)

y_tst_bias1c_20 = np.array(y_tst_bias1c_20)

**Keyword = 50**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1c_50 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
       
        for w2 in ner_keyword_list1_50:
          try:
            counter += model.wv.similarity(w1, w2)
            averager += 1
          except: 
            averager -= 1
            pass
        
        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1c_50.append(counter)

X_val_biased_1c_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias1c_50[j]*(item*0.8))

    X_val_biased_1c_50.append(temp_num_50)
X_val_biased_1c_50 = np.array(X_val_biased_1c_50)

X_test_biased_1c_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias1c_50[j]*(item*0.8))

    X_test_biased_1c_50.append(temp_num_50)
X_test_biased_1c_50 = np.array(X_test_biased_1c_50)

X_val_bias1c_50 = X_val_biased_1c_50
y_val_bias1c_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1c_50.append(class_made)

y_val_bias1c_50 = np.array(y_val_bias1c_50)

# for testing
X_tst_bias1c_50 = X_test_biased_1c_50
y_tst_bias1c_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1c_50.append(class_made)

y_tst_bias1c_50 = np.array(y_tst_bias1c_50)

**Keyword = 100**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1c_100 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0

        for w2 in ner_keyword_list1_100:
          try:
            counter += model.wv.similarity(w1, w2)
            averager += 1
          except: 
            averager -= 1
            pass
        
        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1c_100.append(counter)

X_val_biased_1c_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias1c_100[j]*(item*0.8))

    X_val_biased_1c_100.append(temp_num_100)
X_val_biased_1c_100 = np.array(X_val_biased_1c_100)

X_test_biased_1c_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias1c_100[j]*(item*0.8))

    X_test_biased_1c_100.append(temp_num_100)
X_test_biased_1c_100 = np.array(X_test_biased_1c_100)

X_val_bias1c_100 = X_val_biased_1c_100
y_val_bias1c_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1c_100.append(class_made)

y_val_bias1c_100 = np.array(y_val_bias1c_100)

# for testing
X_tst_bias1c_100 = X_test_biased_1c_100
y_tst_bias1c_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1c_100.append(class_made)

y_tst_bias1c_100 = np.array(y_tst_bias1c_100)

**Keyword = 200**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1c_200 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0

        for w2 in ner_keyword_list1_200:
          try:
            counter += model.wv.similarity(w1, w2)
            averager += 1
          except: 
            averager -= 1
            pass
        
        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1c_200.append(counter)

X_val_biased_1c_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias1c_200[j]*(item*0.8))

    X_val_biased_1c_200.append(temp_num_200)
X_val_biased_1c_200 = np.array(X_val_biased_1c_200)

X_test_biased_1c_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias1c_200[j]*(item*0.8))

    X_test_biased_1c_200.append(temp_num_200)
X_test_biased_1c_200 = np.array(X_test_biased_1c_200)

X_val_bias1c_200 = X_val_biased_1c_200
y_val_bias1c_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1c_200.append(class_made)

y_val_bias1c_200 = np.array(y_val_bias1c_200)

# for testing
X_tst_bias1c_200 = X_test_biased_1c_200
y_tst_bias1c_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1c_200.append(class_made)

y_tst_bias1c_200 = np.array(y_tst_bias1c_200)

**Keyword = 500**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1c_500 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0

        for w2 in ner_keyword_list1_500:
          try:
            counter += model.wv.similarity(w1, w2)
            averager += 1
          except: 
            averager -= 1
            pass
        
        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1c_500.append(counter)

X_val_biased_1c_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias1c_500[j]*(item*0.8))

    X_val_biased_1c_500.append(temp_num_500)
X_val_biased_1c_500 = np.array(X_val_biased_1c_500)

X_test_biased_1c_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias1c_500[j]*(item*0.8))

    X_test_biased_1c_500.append(temp_num_500)
X_test_biased_1c_500 = np.array(X_test_biased_1c_500)

X_val_bias1c_500 = X_val_biased_1c_500
y_val_bias1c_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1c_500.append(class_made)

y_val_bias1c_500 = np.array(y_val_bias1c_500)

# for testing
X_tst_bias1c_500 = X_test_biased_1c_500
y_tst_bias1c_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1c_500.append(class_made)

y_tst_bias1c_500 = np.array(y_tst_bias1c_500)

**Keyword = 750**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1c_750 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0

        for w2 in ner_keyword_list1_750:
          try:
            counter += model.wv.similarity(w1, w2)
            averager += 1
          except: 
            averager -= 1
            pass
        
        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1c_750.append(counter)

X_val_biased_1c_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias1c_750[j]*(item*0.8))

    X_val_biased_1c_750.append(temp_num_750)
X_val_biased_1c_750 = np.array(X_val_biased_1c_750)

X_test_biased_1c_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias1c_750[j]*(item*0.8))

    X_test_biased_1c_750.append(temp_num_750)
X_test_biased_1c_750 = np.array(X_test_biased_1c_750)

X_val_bias1c_750 = X_val_biased_1c_750
y_val_bias1c_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1c_750.append(class_made)

y_val_bias1c_750 = np.array(y_val_bias1c_750)

# for testing
X_tst_bias1c_750 = X_test_biased_1c_750
y_tst_bias1c_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1c_750.append(class_made)

y_tst_bias1c_750 = np.array(y_tst_bias1c_750)

**Keyword = 1000**

In [ ]:
# here I start by calculating our bias multiplier matrix
X_bias1c_1000 = []

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0

        for w2 in ner_keyword_list1_1000:
          try:
            counter += model.wv.similarity(w1, w2)
            averager += 1
          except: 
            averager -= 1
            pass
        
        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1c_1000.append(counter)

X_val_biased_1c_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias1c_1000[j]*(item*0.8))

    X_val_biased_1c_1000.append(temp_num_1000)
X_val_biased_1c_1000 = np.array(X_val_biased_1c_1000)

X_test_biased_1c_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias1c_1000[j]*(item*0.8))

    X_test_biased_1c_1000.append(temp_num_1000)
X_test_biased_1c_1000 = np.array(X_test_biased_1c_1000)

X_val_bias1c_1000 = X_val_biased_1c_1000
y_val_bias1c_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1c_1000.append(class_made)

y_val_bias1c_1000 = np.array(y_val_bias1c_1000)

# for testing
X_tst_bias1c_1000 = X_test_biased_1c_1000
y_tst_bias1c_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1c_1000.append(class_made)

y_tst_bias1c_1000 = np.array(y_tst_bias1c_1000)

## **SVM BIAS 1 NER - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validation = X_val_bias1c_20, X_val_bias1c_50, X_val_bias1c_100, X_val_bias1c_200, X_val_bias1c_500, X_val_bias1c_750, X_val_bias1c_1000
y_validation = y_val_bias1c_20, y_val_bias1c_50, y_val_bias1c_100, y_val_bias1c_200, y_val_bias1c_500, y_val_bias1c_750, y_val_bias1c_1000

X_tst_bias1c = X_tst_bias1c_20, X_tst_bias1c_50, X_tst_bias1c_100, X_tst_bias1c_200, X_tst_bias1c_500, X_tst_bias1c_750, X_tst_bias1c_1000
y_tst_bias1c = y_tst_bias1c_20, y_tst_bias1c_50, y_tst_bias1c_100, y_tst_bias1c_200, y_tst_bias1c_500, y_tst_bias1c_750, y_tst_bias1c_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validation, y_validation, X_tst_bias1c, y_tst_bias1c):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier10 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier10.fit(X_train, y_train)

# validation
    print("SVM validation bias1 - NER", "SVM_C = ", c, "number = ", count)
    y_pred2 = classifier10.predict(X_test)
    print(confusion_matrix(y_test, y_pred2))
    print()
    print(classification_report(y_test, y_pred2))
    print()
    print(precision_recall_fscore_support(y_test, y_pred2, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred2, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred2))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias1 - NER", "SVM_C = ", c, "number = ", count)
    y_pred2_test = classifier10.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred2_test))
    print()
    print(classification_report(a, y_pred2_test))
    print()
    print(precision_recall_fscore_support(a, y_pred2_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred2_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred2_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred2_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")

**BIAS 2 - NER (CoNLL-2003)**

**Keyword = 20**

In [ ]:
# now to bias the X for bias 2
X_bias2c_20 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_20:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2c_20.append(counter)

X_val_biased_2c_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias2c_20[j]*(item*0.8))

    X_val_biased_2c_20.append(temp_num_20)
X_val_biased_2c_20 = np.array(X_val_biased_2c_20)

X_test_biased_2c_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias2c_20[j]*(item*0.8))

    X_test_biased_2c_20.append(temp_num_20)
X_test_biased_2c_20 = np.array(X_test_biased_2c_20)

X_val_bias2c_20 = X_val_biased_2c_20
y_val_bias2c_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2c_20.append(class_made)

y_val_bias2c_20 = np.array(y_val_bias2c_20)

X_tst_bias2c_20 = X_test_biased_2c_20
y_tst_bias2c_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2c_20.append(class_made)

y_tst_bias2c_20 = np.array(y_tst_bias2c_20)


**Keyword = 50**

In [ ]:
# now to bias the X for bias 2
X_bias2c_50 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_50:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2c_50.append(counter)

X_val_biased_2c_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias2c_50[j]*(item*0.8))

    X_val_biased_2c_50.append(temp_num_50)
X_val_biased_2c_50 = np.array(X_val_biased_2c_50)

X_test_biased_2c_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias2c_50[j]*(item*0.8))

    X_test_biased_2c_50.append(temp_num_50)
X_test_biased_2c_50 = np.array(X_test_biased_2c_50)

X_val_bias2c_50 = X_val_biased_2c_50
y_val_bias2c_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2c_50.append(class_made)

y_val_bias2c_50 = np.array(y_val_bias2c_50)

X_tst_bias2c_50 = X_test_biased_2c_50
y_tst_bias2c_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2c_50.append(class_made)

y_tst_bias2c_50 = np.array(y_tst_bias2c_50)



**Keyword = 100**

In [ ]:
# now to bias the X for bias 2
X_bias2c_100 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_100:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2c_100.append(counter)

X_val_biased_2c_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias2c_100[j]*(item*0.8))

    X_val_biased_2c_100.append(temp_num_100)
X_val_biased_2c_100 = np.array(X_val_biased_2c_100)

X_test_biased_2c_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias2c_100[j]*(item*0.8))

    X_test_biased_2c_100.append(temp_num_100)
X_test_biased_2c_100 = np.array(X_test_biased_2c_100)

X_val_bias2c_100 = X_val_biased_2c_100
y_val_bias2c_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2c_100.append(class_made)

y_val_bias2c_100 = np.array(y_val_bias2c_100)

X_tst_bias2c_100 = X_test_biased_2c_100
y_tst_bias2c_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2c_100.append(class_made)

y_tst_bias2c_100 = np.array(y_tst_bias2c_100)



**Keyword = 200**

In [ ]:
# now to bias the X for bias 2
X_bias2c_200 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_200:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2c_200.append(counter)

X_val_biased_2c_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias2c_200[j]*(item*0.8))

    X_val_biased_2c_200.append(temp_num_200)
X_val_biased_2c_200 = np.array(X_val_biased_2c_200)

X_test_biased_2c_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias2c_200[j]*(item*0.8))

    X_test_biased_2c_200.append(temp_num_200)
X_test_biased_2c_200 = np.array(X_test_biased_2c_200)

X_val_bias2c_200 = X_val_biased_2c_200
y_val_bias2c_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2c_200.append(class_made)

y_val_bias2c_200 = np.array(y_val_bias2c_200)

X_tst_bias2c_200 = X_test_biased_2c_200
y_tst_bias2c_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2c_200.append(class_made)

y_tst_bias2c_200 = np.array(y_tst_bias2c_200)



**Keyword = 500**

In [ ]:
# now to bias the X for bias 2
X_bias2c_500 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_500:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2c_500.append(counter)

X_val_biased_2c_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias2c_500[j]*(item*0.8))

    X_val_biased_2c_500.append(temp_num_500)
X_val_biased_2c_500 = np.array(X_val_biased_2c_500)

X_test_biased_2c_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias2c_500[j]*(item*0.8))

    X_test_biased_2c_500.append(temp_num_500)
X_test_biased_2c_500 = np.array(X_test_biased_2c_500)

X_val_bias2c_500 = X_val_biased_2c_500
y_val_bias2c_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2c_500.append(class_made)

y_val_bias2c_500 = np.array(y_val_bias2c_500)

X_tst_bias2c_500 = X_test_biased_2c_500
y_tst_bias2c_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2c_500.append(class_made)

y_tst_bias2c_500 = np.array(y_tst_bias2c_500)



**Keyword = 750**

In [ ]:
# now to bias the X for bias 2
X_bias2c_750 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_750:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2c_750.append(counter)

X_val_biased_2c_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias2c_750[j]*(item*0.8))

    X_val_biased_2c_750.append(temp_num_750)
X_val_biased_2c_750 = np.array(X_val_biased_2c_750)

X_test_biased_2c_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias2c_750[j]*(item*0.8))

    X_test_biased_2c_750.append(temp_num_750)
X_test_biased_2c_750 = np.array(X_test_biased_2c_750)

X_val_bias2c_750 = X_val_biased_2c_750
y_val_bias2c_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2c_750.append(class_made)

y_val_bias2c_750 = np.array(y_val_bias2c_750)

X_tst_bias2c_750 = X_test_biased_2c_750
y_tst_bias2c_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2c_750.append(class_made)

y_tst_bias2c_750 = np.array(y_tst_bias2c_750)



**Keyword = 1000**

In [ ]:
# now to bias the X for bias 2
X_bias2c_1000 = []

# initialise and train the model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=10, iter=10)

# create a list with bias multipliers for each of the words in the TF-IDF vocab, now with wordlist 2
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_1000:
            try:
                counter += model.wv.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
          counter = counter/averager
        except:
          counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias2c_1000.append(counter)

X_val_biased_2c_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias2c_1000[j]*(item*0.8))

    X_val_biased_2c_1000.append(temp_num_1000)
X_val_biased_2c_1000 = np.array(X_val_biased_2c_1000)

X_test_biased_2c_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias2c_1000[j]*(item*0.8))

    X_test_biased_2c_1000.append(temp_num_1000)
X_test_biased_2c_1000 = np.array(X_test_biased_2c_1000)

X_val_bias2c_1000 = X_val_biased_2c_1000
y_val_bias2c_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias2c_1000.append(class_made)

y_val_bias2c_1000 = np.array(y_val_bias2c_1000)

X_tst_bias2c_1000 = X_test_biased_2c_1000
y_tst_bias2c_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias2c_1000.append(class_made)

y_tst_bias2c_1000 = np.array(y_tst_bias2c_1000)



## **SVM BIAS 2 NER - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validation = X_val_bias2c_20, X_val_bias2c_50, X_val_bias2c_100, X_val_bias2c_200, X_val_bias2c_500, X_val_bias2c_750, X_val_bias2c_1000
y_validation = y_val_bias2c_20, y_val_bias2c_50, y_val_bias2c_100, y_val_bias2c_200, y_val_bias2c_500, y_val_bias2c_750, y_val_bias2c_1000

X_tst_bias2 = X_tst_bias2c_20, X_tst_bias2c_50, X_tst_bias2c_100, X_tst_bias2c_200, X_tst_bias2c_500, X_tst_bias2c_750, X_tst_bias2c_1000
y_tst_bias2 = y_tst_bias2c_20, y_tst_bias2c_50, y_tst_bias2c_100, y_tst_bias2c_200, y_tst_bias2c_500, y_tst_bias2c_750, y_tst_bias2c_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validation, y_validation, X_tst_bias2, y_tst_bias2):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier3 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier3.fit(X_train, y_train)

# validation
    print("SVM validation bias2", "SVM_C = ", c, "number = ", count)
    y_pred3 = classifier3.predict(X_test)
    print(confusion_matrix(y_test, y_pred3))
    print()
    print(classification_report(y_test, y_pred3))
    print()
    print(precision_recall_fscore_support(y_test, y_pred3, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred3, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred3))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias2", "SVM_C = ", c, "number = ", count)
    y_pred3_test = classifier3.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred3_test))
    print()
    print(classification_report(a, y_pred3_test))
    print()
    print(precision_recall_fscore_support(a, y_pred3_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred3_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred3_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred3_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")

**BIAS 3 - NER (CoNLL-2003)**

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# load the pretrained file GLove for word similairty
glove_file = datapath('/content/drive/My Drive/Colab Notebooks/Thesis Project/Datasets GloVe/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile('glove.6B.100d.txt')

glove2word2vec(glove_file,word2vec_glove_file)

**Keyword = 20**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3c_20 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_20:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3c_20.append(counter)

X_val_biased_3c_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias3c_20[j]*(item*0.8))

    X_val_biased_3c_20.append(temp_num_20)
X_val_biased_3c_20 = np.array(X_val_biased_3c_20)

X_test_biased_3c_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias3c_20[j]*(item*0.8))

    X_test_biased_3c_20.append(temp_num_20)
X_test_biased_3c_20 = np.array(X_test_biased_3c_20)

X_val_bias3c_20 = X_val_biased_3c_20
y_val_bias3c_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3c_20.append(class_made)

y_val_bias3c_20 = np.array(y_val_bias3c_20)

X_tst_bias3c_20 = X_test_biased_3c_20
y_tst_bias3c_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3c_20.append(class_made)

y_tst_bias3c_20 = np.array(y_tst_bias3c_20)

**Keyword = 50**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3c_50 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_50:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3c_50.append(counter)

X_val_biased_3c_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias3c_50[j]*(item*0.8))

    X_val_biased_3c_50.append(temp_num_50)
X_val_biased_3c_50 = np.array(X_val_biased_3c_50)

X_test_biased_3c_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias3c_50[j]*(item*0.8))

    X_test_biased_3c_50.append(temp_num_50)
X_test_biased_3c_50 = np.array(X_test_biased_3c_50)

X_val_bias3c_50 = X_val_biased_3c_50
y_val_bias3c_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3c_50.append(class_made)

y_val_bias3c_50 = np.array(y_val_bias3c_50)

X_tst_bias3c_50 = X_test_biased_3c_50
y_tst_bias3c_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3c_50.append(class_made)

y_tst_bias3c_50 = np.array(y_tst_bias3c_50)

**Keyword = 100**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3c_100 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_100:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3c_100.append(counter)

X_val_biased_3c_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias3c_100[j]*(item*0.8))

    X_val_biased_3c_100.append(temp_num_100)
X_val_biased_3c_100 = np.array(X_val_biased_3c_100)

X_test_biased_3c_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias3c_100[j]*(item*0.8))

    X_test_biased_3c_100.append(temp_num_100)
X_test_biased_3c_100 = np.array(X_test_biased_3c_100)

X_val_bias3c_100 = X_val_biased_3c_100
y_val_bias3c_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3c_100.append(class_made)

y_val_bias3c_100 = np.array(y_val_bias3c_100)

X_tst_bias3c_100 = X_test_biased_3c_100
y_tst_bias3c_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3c_100.append(class_made)

y_tst_bias3c_100 = np.array(y_tst_bias3c_100)

**Keyword = 200**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3c_200 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_200:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3c_200.append(counter)

X_val_biased_3c_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias3c_200[j]*(item*0.8))

    X_val_biased_3c_200.append(temp_num_200)
X_val_biased_3c_200 = np.array(X_val_biased_3c_200)

X_test_biased_3c_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias3c_200[j]*(item*0.8))

    X_test_biased_3c_200.append(temp_num_200)
X_test_biased_3c_200 = np.array(X_test_biased_3c_200)

X_val_bias3c_200 = X_val_biased_3c_200
y_val_bias3c_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3c_200.append(class_made)

y_val_bias3c_200 = np.array(y_val_bias3c_200)

X_tst_bias3c_200 = X_test_biased_3c_200
y_tst_bias3c_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3c_200.append(class_made)

y_tst_bias3c_200 = np.array(y_tst_bias3c_200)

**Keyword = 500**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3c_500 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_500:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3c_500.append(counter)

X_val_biased_3c_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias3c_500[j]*(item*0.8))

    X_val_biased_3c_500.append(temp_num_500)
X_val_biased_3c_500 = np.array(X_val_biased_3c_500)

X_test_biased_3c_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias3c_500[j]*(item*0.8))

    X_test_biased_3c_500.append(temp_num_500)
X_test_biased_3c_500 = np.array(X_test_biased_3c_500)

X_val_bias3c_500 = X_val_biased_3c_500
y_val_bias3c_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3c_500.append(class_made)

y_val_bias3c_500 = np.array(y_val_bias3c_500)

X_tst_bias3c_500 = X_test_biased_3c_500
y_tst_bias3c_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3c_500.append(class_made)

y_tst_bias3c_500 = np.array(y_tst_bias3c_500)

**Keyword = 750**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3c_750 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_750:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3c_750.append(counter)

X_val_biased_3c_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias3c_750[j]*(item*0.8))

    X_val_biased_3c_750.append(temp_num_750)
X_val_biased_3c_750 = np.array(X_val_biased_3c_750)

X_test_biased_3c_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias3c_750[j]*(item*0.8))

    X_test_biased_3c_750.append(temp_num_750)
X_test_biased_3c_750 = np.array(X_test_biased_3c_750)

X_val_bias3c_750 = X_val_biased_3c_750
y_val_bias3c_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3c_750.append(class_made)

y_val_bias3c_750 = np.array(y_val_bias3c_750)

X_tst_bias3c_750 = X_test_biased_3c_750
y_tst_bias3c_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3c_750.append(class_made)

y_tst_bias3c_750 = np.array(y_tst_bias3c_750)

**Keyword = 1000**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias3c_1000 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list1_1000:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias3c_1000.append(counter)

X_val_biased_3c_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias3c_1000[j]*(item*0.8))

    X_val_biased_3c_1000.append(temp_num_1000)
X_val_biased_3c_1000 = np.array(X_val_biased_3c_1000)

X_test_biased_3c_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias3c_1000[j]*(item*0.8))

    X_test_biased_3c_1000.append(temp_num_1000)
X_test_biased_3c_1000 = np.array(X_test_biased_3c_1000)

X_val_bias3c_1000 = X_val_biased_3c_1000
y_val_bias3c_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias3c_1000.append(class_made)

y_val_bias3c_1000 = np.array(y_val_bias3c_1000)

X_tst_bias3c_1000 = X_test_biased_3c_1000
y_tst_bias3c_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias3c_1000.append(class_made)

y_tst_bias3c_1000 = np.array(y_tst_bias3c_1000)

## **SVM BIAS 3 NER - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validationc = X_val_bias3c_20, X_val_bias3c_50, X_val_bias3c_100, X_val_bias3c_200, X_val_bias3c_500, X_val_bias3c_750, X_val_bias3c_1000
y_validationc = y_val_bias3c_20, y_val_bias3c_50, y_val_bias3c_100, y_val_bias3c_200, y_val_bias3c_500, y_val_bias3c_750, y_val_bias3c_1000

X_tst_bias3c = X_tst_bias3c_20, X_tst_bias3c_50, X_tst_bias3c_100, X_tst_bias3c_200, X_tst_bias3c_500, X_tst_bias3c_750, X_tst_bias3c_1000
y_tst_bias3c = y_tst_bias3c_20, y_tst_bias3c_50, y_tst_bias3c_100, y_tst_bias3c_200, y_tst_bias3c_500, y_tst_bias3c_750, y_tst_bias3c_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validationc, y_validationc, X_tst_bias3c, y_tst_bias3c):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier5 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier5.fit(X_train, y_train)

# validation
    print("SVM validation bias3", "SVM_C = ", c, "number = ", count)
    y_pred4 = classifier5.predict(X_test)
    print(confusion_matrix(y_test, y_pred4))
    print()
    print(classification_report(y_test, y_pred4))
    print()
    print(precision_recall_fscore_support(y_test, y_pred4, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred4, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred4))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias3", "SVM_C = ", c, "number = ", count)
    y_pred4_test = classifier5.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred4_test))
    print()
    print(classification_report(a, y_pred4_test))
    print()
    print(precision_recall_fscore_support(a, y_pred4_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred4_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred4_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred4_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")

**BIAS 4 - NER (CoNLL-2003)**

**Keyword = 20**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias4c_20 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_20:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4c_20.append(counter)

X_val_biased_4c_20 = []
for i,matrixrow in enumerate(X_val):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias4c_20[j]*(item*0.8))

    X_val_biased_4c_20.append(temp_num_20)
X_val_biased_4c_20 = np.array(X_val_biased_4c_20)

X_test_biased_4c_20 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_20 = []
    for j, item in enumerate(matrixrow):

        temp_num_20.append(X_bias4c_20[j]*(item*0.8))

    X_test_biased_4c_20.append(temp_num_20)
X_test_biased_4c_20 = np.array(X_test_biased_4c_20)

X_val_bias4c_20 = X_val_biased_4c_20
y_val_bias4c_20 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4c_20.append(class_made)

y_val_bias4c_20 = np.array(y_val_bias4c_20)

X_tst_bias4c_20 = X_test_biased_4c_20
y_tst_bias4c_20 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4c_20.append(class_made)

y_tst_bias4c_20 = np.array(y_tst_bias4c_20)

**Keyword = 50**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias4c_50 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_50:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4c_50.append(counter)

X_val_biased_4c_50 = []
for i,matrixrow in enumerate(X_val):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias4c_50[j]*(item*0.8))

    X_val_biased_4c_50.append(temp_num_50)
X_val_biased_4c_50 = np.array(X_val_biased_4c_50)

X_test_biased_4c_50 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_50 = []
    for j, item in enumerate(matrixrow):

        temp_num_50.append(X_bias4c_50[j]*(item*0.8))

    X_test_biased_4c_50.append(temp_num_50)
X_test_biased_4c_50 = np.array(X_test_biased_4c_50)

X_val_bias4c_50 = X_val_biased_4c_50
y_val_bias4c_50 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4c_50.append(class_made)

y_val_bias4c_50 = np.array(y_val_bias4c_50)

X_tst_bias4c_50 = X_test_biased_4c_50
y_tst_bias4c_50 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4c_50.append(class_made)

y_tst_bias4c_50 = np.array(y_tst_bias4c_50)

**Keyword = 100**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias4c_100 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_100:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4c_100.append(counter)

X_val_biased_4c_100 = []
for i,matrixrow in enumerate(X_val):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias4c_100[j]*(item*0.8))

    X_val_biased_4c_100.append(temp_num_100)
X_val_biased_4c_100 = np.array(X_val_biased_4c_100)

X_test_biased_4c_100 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_100 = []
    for j, item in enumerate(matrixrow):

        temp_num_100.append(X_bias4c_100[j]*(item*0.8))

    X_test_biased_4c_100.append(temp_num_100)
X_test_biased_4c_100 = np.array(X_test_biased_4c_100)

X_val_bias4c_100 = X_val_biased_4c_100
y_val_bias4c_100 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4c_100.append(class_made)

y_val_bias4c_100 = np.array(y_val_bias4c_100)

X_tst_bias4c_100 = X_test_biased_4c_100
y_tst_bias4c_100 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4c_100.append(class_made)

y_tst_bias4c_100 = np.array(y_tst_bias4c_100)

**Keyword = 200**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias4c_200 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_200:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4c_200.append(counter)

X_val_biased_4c_200 = []
for i,matrixrow in enumerate(X_val):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias4c_200[j]*(item*0.8))

    X_val_biased_4c_200.append(temp_num_200)
X_val_biased_4c_200 = np.array(X_val_biased_4c_200)

X_test_biased_4c_200 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_200 = []
    for j, item in enumerate(matrixrow):

        temp_num_200.append(X_bias4c_200[j]*(item*0.8))

    X_test_biased_4c_200.append(temp_num_200)
X_test_biased_4c_200 = np.array(X_test_biased_4c_200)

X_val_bias4c_200 = X_val_biased_4c_200
y_val_bias4c_200 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4c_200.append(class_made)

y_val_bias4c_200 = np.array(y_val_bias4c_200)

X_tst_bias4c_200 = X_test_biased_4c_200
y_tst_bias4c_200 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4c_200.append(class_made)

y_tst_bias4c_200 = np.array(y_tst_bias4c_200)

**Keyword = 500**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias4c_500 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_500:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4c_500.append(counter)

X_val_biased_4c_500 = []
for i,matrixrow in enumerate(X_val):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias4c_500[j]*(item*0.8))

    X_val_biased_4c_500.append(temp_num_500)
X_val_biased_4c_500 = np.array(X_val_biased_4c_500)

X_test_biased_4c_500 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_500 = []
    for j, item in enumerate(matrixrow):

        temp_num_500.append(X_bias4c_500[j]*(item*0.8))

    X_test_biased_4c_500.append(temp_num_500)
X_test_biased_4c_500 = np.array(X_test_biased_4c_500)

X_val_bias4c_500 = X_val_biased_4c_500
y_val_bias4c_500 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4c_500.append(class_made)

y_val_bias4c_500 = np.array(y_val_bias4c_500)

X_tst_bias4c_500 = X_test_biased_4c_500
y_tst_bias4c_500 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4c_500.append(class_made)

y_tst_bias4c_500 = np.array(y_tst_bias4c_500)

**Keyword = 750**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias4c_750 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_750:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4c_750.append(counter)

X_val_biased_4c_750 = []
for i,matrixrow in enumerate(X_val):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias4c_750[j]*(item*0.8))

    X_val_biased_4c_750.append(temp_num_750)
X_val_biased_4c_750 = np.array(X_val_biased_4c_750)

X_test_biased_4c_750 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_750 = []
    for j, item in enumerate(matrixrow):

        temp_num_750.append(X_bias4c_750[j]*(item*0.8))

    X_test_biased_4c_750.append(temp_num_750)
X_test_biased_4c_750 = np.array(X_test_biased_4c_750)

X_val_bias4c_750 = X_val_biased_4c_750
y_val_bias4c_750 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4c_750.append(class_made)

y_val_bias4c_750 = np.array(y_val_bias4c_750)

X_tst_bias4c_750 = X_test_biased_4c_750
y_tst_bias4c_750 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4c_750.append(class_made)

y_tst_bias4c_750 = np.array(y_tst_bias4c_750)

**Keyword = 1000**

In [ ]:
# initialise and train the model
model2 = KeyedVectors.load_word2vec_format(word2vec_glove_file)

# create biases in the same fashion as before
X_bias4c_1000 = []
for w1 in tf_idf_feature_names_training:
        counter = 0
        averager = 0
        for w2 in ner_keyword_list2_1000:
            try:
                counter += model2.similarity(w1, w2)
                averager += 1
            except:
                averager -= 1
                pass

        try:
            counter = counter/averager
        except:
            counter = counter
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias4c_1000.append(counter)

X_val_biased_4c_1000 = []
for i,matrixrow in enumerate(X_val):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias4c_1000[j]*(item*0.8))

    X_val_biased_4c_1000.append(temp_num_1000)
X_val_biased_4c_1000 = np.array(X_val_biased_4c_1000)

X_test_biased_4c_1000 = []
for i,matrixrow in enumerate(X_tst):
    temp_num_1000 = []
    for j, item in enumerate(matrixrow):

        temp_num_1000.append(X_bias4c_1000[j]*(item*0.8))

    X_test_biased_4c_1000.append(temp_num_1000)
X_test_biased_4c_1000 = np.array(X_test_biased_4c_1000)

X_val_bias4c_1000 = X_val_biased_4c_1000
y_val_bias4c_1000 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias4c_1000.append(class_made)

y_val_bias4c_1000 = np.array(y_val_bias4c_1000)

X_tst_bias4c_1000 = X_test_biased_4c_1000
y_tst_bias4c_1000 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias4c_1000.append(class_made)

y_tst_bias4c_1000 = np.array(y_tst_bias4c_1000)

## **SVM BIAS 4 NER - Keywords 20, 50, 100, 200, 500, 750, 1000 - Max-feat = 4000**

In [ ]:
X_validation = X_val_bias4c_20, X_val_bias4c_50, X_val_bias4c_100, X_val_bias4c_200, X_val_bias4c_500, X_val_bias4c_750, X_val_bias4c_1000
y_validation = y_val_bias4c_20, y_val_bias4c_50, y_val_bias4c_100, y_val_bias4c_200, y_val_bias4c_500, y_val_bias4c_750, y_val_bias4c_1000

X_tst_bias4 = X_tst_bias4c_20, X_tst_bias4c_50, X_tst_bias4c_100, X_tst_bias4c_200, X_tst_bias4c_500, X_tst_bias4c_750, X_tst_bias4c_1000
y_tst_bias4 = y_tst_bias4c_20, y_tst_bias4c_50, y_tst_bias4c_100, y_tst_bias4c_200, y_tst_bias4c_500, y_tst_bias4c_750, y_tst_bias4c_1000

In [ ]:
# train test split for validation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_performing_fscore, best_performing_c, best_performing_key = 0, None, None
fscore = 0
count = 0

for x, y, z, a in zip(X_validation, y_validation, X_tst_bias4, y_tst_bias4):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
  for c in SVM_C:
    classifier6 = svm.SVC(C=c, kernel='linear', degree=7, gamma='auto', random_state=0)
    classifier6.fit(X_train, y_train)

# validation
    print("SVM validation bias4", "SVM_C = ", c, "number = ", count)
    y_pred5 = classifier6.predict(X_test)
    print(confusion_matrix(y_test, y_pred5))
    print()
    print(classification_report(y_test, y_pred5))
    print()
    print(precision_recall_fscore_support(y_test, y_pred5, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred5, average='macro')[2])
    print()
    print(accuracy_score(y_test, y_pred5))

# testing

    #for z, a in X_tst_bias1, y_tst_bias1:
    print("SVM test bias4", "SVM_C = ", c, "number = ", count)
    y_pred5_test = classifier6.predict(z)
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    print(confusion_matrix(a, y_pred5_test))
    print()
    print(classification_report(a, y_pred5_test))
    print()
    print(precision_recall_fscore_support(a, y_pred5_test, average='macro'))
    print(precision_recall_fscore_support(a, y_pred5_test, average='macro')[2])
    fscore = (precision_recall_fscore_support(a, y_pred5_test, average='macro')[2])
    print()
    print(accuracy_score(a, y_pred5_test))

    if best_performing_fscore < fscore: 
      best_performing_fscore, best_performing_c, best_performing_key = fscore, c, count
    count+= 1

print()
print("the best performing C parameter is: ", best_performing_c, "\n", "the best performing keyword number is: ", best_performing_key, "\n", 
      "with an f1-score of :", best_performing_fscore, "\n", "for max_feat = 4000")